In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm import tqdm
import datetime
import proplot as plot
from wrf import getvar
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

from dask.distributed import Client
client = Client(processes=True, threads_per_worker=1,n_workers=2)
import time
import numpy as np

/tmp/ipykernel_240536/918310841.py:22: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/miniconda3/envs/fred_workenv/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42847 instead
  warnings.warn(


In [2]:
from sklearn.decomposition import PCA
def smooth_array_along_axis(array=None,axis=None,window=3):
    if axis==0:
        aN = array.copy()
    else:
        aN = np.swapaxes(array,axis,0)
    aN = aN.reshape((aN.shape[0], -1))
    return np.asarray([np.convolve(aN[:,i],np.ones(window)/window,'same')[0:aN.shape[0]] for i in range(len(aN[0,:]))]).transpose()

def dummy(inlist=None):
    return inlist

def do_PCA(array=None):
    skpcaVAR = PCA()
    skpcaVAR.fit(array)
    return skpcaVAR

In [3]:
def forward_diff(arrayin=None,delta=None,axis=None,LT=1):
    result = []
    if axis==0:
            for i in range(0,arrayin.shape[axis]-LT):
                temp = (arrayin[i+LT,:]-arrayin[i,:])/(LT*delta)
                result.append(temp)
            return np.asarray(result)

In [ ]:
ctl_du,ctl_dv,ctl_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                         '/scratch/06040/tg853394/tc/output/redux/maria/','ctl')
ncrf_36h_du,ncrf_36h_dv,ncrf_36h_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                                        '/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_36h')
ncrf_60h_du,ncrf_60h_dv,ncrf_60h_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                                        '/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_60h')
lwcrf_du,lwcrf_dv,lwcrf_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                               '/scratch/06040/tg853394/tc/output/redux/maria/','lwcrf')

ctl_dict = {"du":ctl_du,"dv":ctl_dv,"dw":ctl_dw}
del ctl_du,ctl_dv,ctl_dw
gc.collect()
ncrf_36h_dict = {"du":ncrf_36h_du,"dv":ncrf_36h_dv,"dw":ncrf_36h_dw}
del ncrf_36h_du,ncrf_36h_dv,ncrf_36h_dw
gc.collect()
ncrf_60h_dict = {"du":ncrf_60h_du,"dv":ncrf_60h_dv,"dw":ncrf_60h_dw}
del ncrf_60h_du,ncrf_60h_dv,ncrf_60h_dw
gc.collect()
lwcrf_dict = {"du":lwcrf_du,"dv":lwcrf_dv,"dw":lwcrf_dw}
del lwcrf_du,lwcrf_dv,lwcrf_dw
gc.collect()

In [7]:
import importlib
importlib.reload(preproc_maria)
############################################################################################################################################################
# uvw-heatsum
############################################################################################################################################################
#ctl_outdict,_ = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ctl',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_uvwF(smooth=True)
#preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ncrf_36h',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_uvwF(smooth=True)
#preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ncrf_60h',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_uvwF(smooth=True)
#preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','lwcrf',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_uvwF(smooth=True)
############################################################################################################################################################
# dudvdw
############################################################################################################################################################
#dudvdw_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ctl',5,'new').preproc_dudvdw()
############################################################################################################################################################
# dthdQ
#dthdq_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ctl',5,'new').preproc_dthdQ()
############################################################################################################################################################
############################################################################################################################################################
# Smoothed and flattened theta
############################################################################################################################################################
#ctl_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ctl',5,'new').preproc_onevar('theta','orig',None)
#ncrf36_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ncrf_36h',5,'new').preproc_onevar('theta','orig',None)
#ncrf60_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ncrf_60h',5,'new').preproc_onevar('theta','orig',None)
#lwcrf_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','lwcrf',5,'new').preproc_onevar('theta','orig',None)
#theta_outdict = {'ctlTHETA':ctl_outdict['theta'],'ncrf36THETA':ncrf36_outdict['theta'],'ncrf60THETA':ncrf60_outdict['theta'],\
#                 'lwcrfTHETA':lwcrf_outdict['theta']}
############################################################################################################################################################
ctl_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ctl',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_heateq('IR')
ncrf36_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ncrf_36h',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_heateq('IR')
ncrf60_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','ncrf_60h',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_heateq('IR')
lwcrf_outdict = preproc_maria.preprocess('/scratch/itam/maria/processed/','/scratch/itam/maria/','lwcrf',5,'new',True,[3,0,0,0],'preproc_dict1b_g').preproc_heateq('IR')
ir_outdict = {'ctlIR':ctl_outdict['HEAT'],'ncrf36IR':ncrf36_outdict['HEAT'],'ncrf60IR':ncrf60_outdict['HEAT'],'lwcrfIR':lwcrf_outdict['HEAT']}
noir_outdict = {'ctlNOIR':ctl_outdict['NOHEAT'],'ncrf36NOIR':ncrf36_outdict['NOHEAT'],'ncrf60NOIR':ncrf60_outdict['NOHEAT'],'lwcrfNOIR':lwcrf_outdict['NOHEAT']}

---Finish!---
---Finish!---
---Finish!---
---Finish!---


In [8]:
expname = 'IR'
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/'
read_and_proc.save_to_pickle(folderpath+'pca/output/uvwheat/'+str(expname)+'_'+'preproc_dict1_g_onevar',ir_outdict,'PICKLE')

expname = 'NOIR'
read_and_proc.save_to_pickle(folderpath+'pca/output/uvwheat/'+str(expname)+'_'+'preproc_dict1_g_onevar',noir_outdict,'PICKLE')

In [10]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/'
irdict = read_and_proc.depickle(folderpath+'pca/output/flatvar/NOIR_'+'preproc_dict1')

In [4]:
expname = ['ctl','ncrf_60h']
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/'
listdict = [read_and_proc.depickle(folderpath+'pca/output/uvwheat/test/'+str(expname[i])+'_'+'preproc_dict1') for i in range(2)]

In [ ]:
fig,axs = plot.subplots(ncols=2,nrows=1,axwidth=2.095,refaspect=0.831)
var = 'NOIR'
axs[0].plot(np.nanmean(irdict['ctlNOIR'][58].reshape(39,360,167),axis=(1,2)),np.linspace(0,38,39))
axs[0].plot(np.nanmean(irdict['ncrf60NOIR'][58].reshape(39,360,167),axis=(1,2)),np.linspace(0,38,39))
axs[0].format(title=str(var)+';T=58hr')
axs[1].plot(np.nanmean(irdict['ctlNOIR'][59].reshape(39,360,167),axis=(1,2)),np.linspace(0,38,39),label='CTRL')
axs[1].plot(np.nanmean(irdict['ncrf60NOIR'][59].reshape(39,360,167),axis=(1,2)),np.linspace(0,38,39),label='NCRF60')
axs[1].format(title=str(var)+';T=59hr')
axs[1].legend(loc='lr',ncols=1)
plt.savefig('./figures/'+str(var)+'_new.png',dpi=200)
plt.show()